In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
import simulate
pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_rows', 100)

In [ ]:
spx = pd.read_csv('^GSPC.csv', index_col=0)

start = dt.date(2020, 1, 1)
end = dt.date(2080, 12, 31)


Market = simulate.Market(spx.iloc[-7500:, -2], start, end)

## Setting up functions to calculate performance

In [ ]:
def nd(s, pv_u, dst, td, cash, phase, g):
    # determine new debt dependent on phase
	# dont aquire debt or repay debt in phase 3 or 4
    if phase == 3 or phase == 4:
        return 0, s
    
    if phase == 1:
        upper_debt_limit = pv_u/(1+g)
        new_debt = min(max(g*s, g*(pv_u-td)), dst-pv_u-s)
        return (new_debt, s)
    
    if phase == 2: 
        stocks_sold = max(pv_u-dst, 0)
        debt_repayment = min(td, s + stocks_sold)
        leftover_savings = max(s-debt_repayment - stocks_sold, 0)
        return -debt_repayment, leftover_savings

In [ ]:
def update_cash(phase, pv_u, tv_u, s, cash, pi_rf, pi_hat):
    if phase <= 3:
        return 0, s
    
    if phase == 4:
        
        desired_cash = (1-pi_rf)*(tv_u+s)
        desired_savings = (pi_rf)*(tv_u+s)
        change_in_stock = desired_savings - pv_u
        
        #check_pi = (pv_u + change_in_stock)/(tv_u+s)
        
        return desired_cash, change_in_stock

In [ ]:
def phase_check(phase, pi_rf, pi_rm, pi_hat, td):
    if phase == 4:
        return 4
    
    if td > 0:
        #has target not been reached?
        if pi_hat < pi_rm and phase <= 1:
            return 1
        else:
            # if target has been reached once and debt remains, stay in phase 2
            return 2
    
    #if target has been reached and no debt remains
    #is the value still above the target?
    if pi_hat < pi_rf:
        return 3
    else:
        return 4

In [ ]:
def calc_pi(gamma, sigma, mr, rate, cost = 0):
    return (mr - cost - rate)/(gamma * sigma)

In [ ]:
update_cash(4, 900, 1000, 50, 100, 0.8333333333333333333, 0.9)

## Combining with investment profile

In [ ]:
savings = pd.read_csv('investment_plan.csv', sep=';', index_col=0)['Earnings'].values

In [ ]:
def calculate_return(savings, market_returns, gearing_cap, pi_rf, pi_rm, rf, rm):
    
    # Running controls
    assert len(savings) == len(market), 'Investment plan should be same no of periods as market' 
    
    # Setting up constants and dataframe for calculation
    ses = savings.sum()     # Possibly add more sophisticated discounting
    ist = pi_rm*ses
    columns = ['period', 'savings', 'cash', 'new_debt', 'total_debt', 'nip', 'pv_p', 
               'interest', 'market_returns', 'pv_u', 'tv_u', 'dst', 'phase', 'pi_hat', 'ses']
    
    pp = pd.DataFrame(np.empty((len(savings), len(columns))), columns = columns)
    
    pp['ses'] = ses
    pp['period'] = range(len(savings))
    pp['market_returns'] = market_returns
    pp['savings'] = savings
    pp.loc[0, 'market_returns'] = 0
    
                      
    # Period 0 primo
    pp.loc[0, 'cash'] = 0
    pp.loc[0, 'new_debt'] = pp.loc[0, 'savings']*gearing_cap
    pp.loc[0, 'total_debt'] = pp.loc[0, 'new_debt']
    pp.loc[0, 'nip'] = pp.loc[0, 'new_debt'] + pp.loc[0, 'savings']
    pp.loc[0, 'pv_p'] = pp.loc[0, 'nip']
    pp.loc[0, 'pi_hat'] =  pp.loc[0, 'pv_p']/ses
    
    # Period 0 ultimo
    pp.loc[0, 'interest'] = pp.loc[0, 'new_debt']*rm                   
    pp.loc[0, 'pv_u'] = pp.loc[0, 'pv_p']                 
    pp.loc[0, 'tv_u'] = pp.loc[0, 'pv_u'] + pp.loc[0, 'cash']
    pp.loc[0, 'dst'] = ist
    pp.loc[0, 'phase'] = 1
    
              
    
    # Looping over all reminaning periods
    for i in range(1, len(savings)):
 

        # Period t > 0 primo
        pp.loc[i, 'cash'] = pp.loc[i-1, 'cash']*(1+rf) 
        pp.loc[i, 'new_debt'], pp.loc[i, 'savings'] = nd(pp.loc[i, 'savings'], pp.loc[i-1, 'pv_u'], 
                                                         pp.loc[i-1, 'dst'], pp.loc[i-1, 'total_debt'],
                                                         pp.loc[i-1, 'cash'], pp.loc[i-1, 'phase'], gearing_cap)
        pp.loc[i, 'total_debt'] = pp.loc[i-1, 'total_debt'] + pp.loc[i, 'new_debt']
        
        pp.loc[i, 'cash'], pp.loc[i, 'savings'] = update_cash(pp.loc[i-1, 'phase'], pp.loc[i-1, 'pv_u'], pp.loc[i-1, 'tv_u'],
                                                                     pp.loc[i, 'savings'], pp.loc[i, 'cash'], pi_rf, pp.loc[i-1, 'pi_hat'])

        pp.loc[i, 'nip'] = pp.loc[i, 'savings'] + max(0, pp.loc[i, 'new_debt'])
        pp.loc[i, 'pv_p'] = pp.loc[i-1, 'pv_u'] + pp.loc[i, 'nip']        
        
        
        # Period t > 0 ultimo
        pp.loc[i, 'interest'] = max(pp.loc[i, 'total_debt']*rm, 0)
        pp.loc[i, 'pv_u'] = pp.loc[i, 'pv_p']*(1+pp.loc[i, 'market_returns'])-pp.loc[i, 'interest']
        pp.loc[i, 'tv_u'] = pp.loc[i, 'pv_u'] + pp.loc[i, 'cash']
        pp.loc[i, 'pi_hat'] = min(pp.loc[i, 'pv_u']/ses, pp.loc[i, 'pv_u']/pp.loc[i, 'tv_u'])
        
        pp.loc[i, 'phase'] = phase_check(pp.loc[i-1, 'phase'], pi_rf, pi_rm, pp.loc[i, 'pi_hat'], pp.loc[i, 'total_debt'])
        target_pi = pi_rm if pp.loc[i-1, 'phase'] < 3 else pi_rf
        pp.loc[i, 'dst'] = max(pp.loc[i, 'tv_u']*target_pi, ist)  # Moving stock target
        #pp.loc[i, 'dst'] = max(pp.loc[i-1, 'dst'], max(pp.loc[i, 'tv_u']*target_pi, ist))  # Moving stock target        

    return pp

In [ ]:
def calculate100return(savings, market_returns):
     # Running controls
    assert len(savings) == len(market), 'Investment plan should be same no of periods as market' 
    
    columns = ['period', 'savings', 'pv_p', 'market_returns', 'tv_u']
        
    pp = pd.DataFrame(np.empty((len(savings), len(columns))), columns = columns)
    
    pp['period'] = range(len(savings))
    pp['market_returns'] = market_returns
    pp['savings'] = savings
    pp.loc[0, 'market_returns'] = 0
    pp.loc[0, 'pv_p'] = pp.loc[0, 'savings']
    pp.loc[0, 'tv_u'] = pp.loc[0, 'savings']
    
    for i in range(1, len(savings)):
 
        # Period t > 0 primo
        pp.loc[i, 'pv_p'] = pp.loc[i-1, 'tv_u'] + pp.loc[i, 'savings']        
        
        # Period t > 0 ultimo
        pp.loc[i, 'tv_u'] = pp.loc[i, 'pv_p']*(1+pp.loc[i, 'market_returns'])
    
    return pp

In [ ]:
def calculate9050return(savings, market_returns, rf):
    # Strategy where 90% of value is initially invested in stocks, rest in risk free asset
    # Ratio of stocks falls linearly to 50% by age 65 and stays there
    
    # Running controls
    assert len(savings) == len(market), 'Investment plan should be same no of periods as market' 
    
    columns = ['period',  'savings', 'cash', 'pv_p', 'market_returns', 'pv_u', 'tv_u', 'ratio']
        
    pp = pd.DataFrame(np.empty((len(savings), len(columns))), columns = columns)

    pp['period'] = range(len(savings))
    pp['market_returns'] = market_returns
    pp['savings'] = savings
    pp.loc[0, 'market_returns'] = 0
    pp.loc[0, 'pv_p'] = pp.loc[0, 'savings']*0.9
    pp.loc[0, 'cash'] = pp.loc[0, 'savings']*0.1
    pp.loc[0, 'pv_u'] = pp.loc[0, 'pv_p']    
    pp.loc[0, 'tv_u'] = pp.loc[0, 'savings']
    pp.loc[0, 'ratio'] = 90
    
    for i in range(1, len(savings)):
        ratio = max(90 - pp.loc[i, 'period'], 50)
        pp.loc[i, 'ratio'] = ratio
        
        # Period t > 0 primo
        pp.loc[i, 'pv_p'] = pp.loc[i-1, 'pv_u'] + pp.loc[i, 'savings']*(ratio/100)        
        pp.loc[i, 'cash'] = pp.loc[i-1, 'cash']*(1+rf) + pp.loc[i, 'savings']*(1-ratio/100)
        
        # Period t > 0 ultimo
        pp.loc[i, 'pv_u'] = pp.loc[i, 'pv_p']*(1+pp.loc[i, 'market_returns'])
        pp.loc[i, 'tv_u'] = pp.loc[i, 'pv_u'] + pp.loc[i, 'cash']
    
    return pp

In [ ]:
investments = savings*0.05

rf = 0.02
rm = 0.03

gamma = 1.5
sigma = 0.20**2
mr = 0.07

pi_rf = calc_pi(gamma, sigma, mr, rf, cost = 0)
pi_rm = calc_pi(gamma, sigma, mr, rm, cost = 0)
print('pi_rf:', pi_rf, ' pi_rm:', pi_rm)

market = Market.norm_innovations(random_state=11).asfreq('BYS', 'pad')

port = calculate_return(investments, market['Price'].pct_change().values, 1, pi_rf, pi_rm, rf, rm)
port100 = calculate100return(investments, market['Price'].pct_change().values)
port9050 = calculate9050return(investments, market['Price'].pct_change().values, rf)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15, 15))
vars_to_plot = ['cash', 'total_debt', 'tv_u', '100tvu', '9050tvu']
port['100tvu'] = port100['tv_u']
port['9050tvu'] = port9050['tv_u']
ax.plot(port.loc[:,vars_to_plot])
ax.legend(vars_to_plot)

## Characteristics of strategies

In [ ]:
std = port.loc[:, ['tv_u', '100tvu', '9050tvu']].pct_change().std()

horizon = Market.years-1
mean_return = (port.loc[horizon, ['tv_u', '100tvu', '9050tvu']]/port.loc[0, ['tv_u', '100tvu', '9050tvu']])**(1/horizon)-1

In [ ]:
mean_return

In [ ]:
std

In [ ]:
sharpe = mean_return/std

In [ ]:
sharpe